<a href="https://colab.research.google.com/github/nataliyah123/ibia-Ersilia/blob/main/StandardizeSmiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install PyTDC

In [ ]:
%%capture

!pip install rdkit-pypi

In [ ]:
from tdc.single_pred import Tox
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors as rd
from rdkit.Chem.MolStandardize import rdMolStandardize

# Standardize

In [ ]:
def standardize(smiles):
  mol = Chem.MolFromSmiles(smiles)
  clean_mol = rdMolStandardize.Cleanup(mol) 
  parent_clean_mol = rdMolStandardize.FragmentParent(clean_mol)
  uncharger = rdMolStandardize.Uncharger() # annoying, but necessary as no convenience method exists
  uncharged_parent_clean_mol = uncharger.uncharge(parent_clean_mol)     
  te = rdMolStandardize.TautomerEnumerator() # idem
  taut_uncharged_parent_clean_mol = te.Canonicalize(uncharged_parent_clean_mol) 
  stdSmiles = Chem.MolToSmiles(taut_uncharged_parent_clean_mol )
  # print('I am uncharged',taut_uncharged_parent_clean_mol)    
  return stdSmiles


# TDC common data

In [ ]:
data = Tox(name = 'DILI')
# print("data",data)
split = data.get_split()
# print("split",split['train'])


Downloading...
100%|██████████| 26.7k/26.7k [00:00<00:00, 1.11MiB/s]
Loading...
Done!


In [ ]:
# pd.DataFrame.from_dict(split['train'])
dictodf = pd.DataFrame(split['train'])
tdcdf2std = []
itemsintrain = len(dictodf)
for row in range(itemsintrain):  
  tdcdf2std.append((standardize(dictodf.iloc[row,1]),dictodf.iloc[row,2]))
  
tdcdf2std= pd.DataFrame(tdcdf2std,columns=['smiles', 'activity'])
tdcdf2std.head()


,smiles,activity
0,C[N+](C)(C)CC(=O)[O-],0.0
1,O=C(NC(CO)C(O)c1ccc([N+](=O)[O-])cc1)C(Cl)Cl,0.0
2,O=C(O)c1ccccc1O,0.0
3,CC(NC(C)(C)C)C(=O)c1cccc(Cl)c1,0.0
4,NCCc1ccc(O)c(O)c1,0.0


In [ ]:
tdcdf2std.shape

(332, 2)

# Dili Dataset

In [ ]:
dilipadel = pd.read_csv('dili_padel_2d.csv')
dilismiles = pd.read_csv('dilismiles.csv', names=['smiles'])
dilies = pd.DataFrame()
dilies['smiles']=dilismiles
dilies['activity'] = dilipadel['Outcome']

In [ ]:
dilismiles.iloc[0]

smiles    [H][C@]1([S-])O[C@]([H])(CO)[C@@]([H])(O)[C@](...
Name: 0, dtype: object

In [ ]:
dilies

,smiles,activity
0,[H][C@]1([S-])O[C@]([H])(CO)[C@@]([H])(O)[C@](...,1
1,CN(CCCCCCCCCCN(C)C(=O)Oc1cccc(c1)[N+](C)(C)C)C...,1
2,[H][C@]12Cc3ccc(OC)cc3[C@@]3(CCCC[C@]13[H])CCN2C,0
3,[H][C@@]12CCCN1C(=O)[C@]([H])(Cc1ccccc1)N1C(=O...,0
4,[H][C@]1(C)C[C@@]2([H])[C@]3([H])CCC4=CC(=O)C=...,0
...,...,...
583,OC1C(O)C(O)C(O)C(O)C1O,0
584,OCC(S)CS,1
585,OCCNCCNc1ccc(NCCNCCO)c2C(=O)c3c(O)ccc(O)c3C(=O...,1
586,[H]C([H])([H])C([H])([H])N(C(=S)SSC(=S)N(C([H]...,1


In [ ]:
dilismiles.head()

,smiles
0,[H][C@]1([S-])O[C@]([H])(CO)[C@@]([H])(O)[C@](...
1,CN(CCCCCCCCCCN(C)C(=O)Oc1cccc(c1)[N+](C)(C)C)C...
2,[H][C@]12Cc3ccc(OC)cc3[C@@]3(CCCC[C@]13[H])CCN2C
3,[H][C@@]12CCCN1C(=O)[C@]([H])(Cc1ccccc1)N1C(=O...
4,[H][C@]1(C)C[C@@]2([H])[C@]3([H])CCC4=CC(=O)C=...


In [ ]:
dilidf2std = []
itemsindilidf = dilies.shape[0]
for row in range(itemsindilidf):  
  dilidf2std.append((standardize(dilies.iloc[row,0]),dilies.iloc[row,1]))
   
dilidf2std= pd.DataFrame(dilidf2std,columns=['smiles', 'activity'])
dilidf2std.head()

,smiles,activity
0,OC[C@H]1O[C@H](S)[C@H](O)[C@@H](O)[C@@H]1O,1
1,CN(CCCCCCCCCCN(C)C(=O)Oc1cccc([N+](C)(C)C)c1)C...,1
2,COc1ccc2c(c1)[C@]13CCCC[C@@H]1[C@H](C2)N(C)CC3,0
3,CN1CC(C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4C(=O...,0
4,C[C@H]1C[C@H]2[C@@H]3CC=C4CC(=O)C=C[C@]4(C)[C@...,0


In [ ]:
df = pd.concat([dilidf2std, tdcdf2std])
df = df.reset_index(drop=True)
df_gpby = df.groupby(list(df.columns))
idx = [x[0] for x in df_gpby.groups.values() if len(x) == 1]
df.reindex(idx)

,smiles,activity
216,Brc1ccccc1,1.0
801,C#CCN(C)C(C)Cc1ccccc1,0.0
117,C#C[C@]1(O)CC[C@H]2[C@@H]3CC=C4CC(=O)CCC4[C@H]...,1.0
124,C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=Cc5oncc5C[C@]4(...,1.0
120,C#C[C@]1(O)CC[C@H]2[C@@H]3CCc4cc(O)ccc4[C@H]3C...,1.0
...,...,...
222,S=C=Nc1ccccc1,1.0
247,S=c1[nH]cnc2[nH]cnc12,1.0
98,[N-]=[N+]=CC(=O)OCC(N)C(=O)O,1.0
223,c1cc[n+]2c(c1)-c1cccc[n+]1CC2,1.0


In [ ]:
dilidf2std, tdcdf2std
print(dilidf2std.columns==tdcdf2std.columns)

[ True  True]


# Common Rows between two Dataframes

In [ ]:
df_merge =dilidf2std.merge(tdcdf2std, how = 'inner' ,indicator=False)
df_merge

,smiles,activity
0,CCOC(=O)Nc1ccc2c(c1)N(C(=O)CCN1CCOCC1)c1ccccc1S2,1
1,CCOC(=O)C1(c2ccccc2)CCN(CCC(C#N)(c2ccccc2)c2cc...,0
2,NNCCc1ccccc1,0
3,C[N+](C)(C)CC(=O)[O-],0
4,NCCCC(O)(P(=O)(O)O)P(=O)(O)O,0
...,...,...
143,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,1
144,O=C(O)CCc1nc(-c2ccccc2)c(-c2ccccc2)o1,1
145,Oc1c(Cl)cc(Cl)cc1Sc1cc(Cl)cc(Cl)c1O,1
146,OC1C(O)C(O)C(O)C(O)C1O,0


# Find Rows Which Are Not common Between Two dataframes

In [ ]:
pd.concat([dilidf2std,tdcdf2std]).drop_duplicates(keep=False)

,smiles,activity
0,OC[C@H]1O[C@H](S)[C@H](O)[C@@H](O)[C@@H]1O,1.0
1,CN(CCCCCCCCCCN(C)C(=O)Oc1cccc([N+](C)(C)C)c1)C...,1.0
2,COc1ccc2c(c1)[C@]13CCCC[C@@H]1[C@H](C2)N(C)CC3,0.0
3,CN1CC(C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4C(=O...,0.0
4,C[C@H]1C[C@H]2[C@@H]3CC=C4CC(=O)C=C[C@]4(C)[C@...,0.0
...,...,...
327,NC1CCCCC1N,1.0
328,C[N+]1(C)C2CCC1CC(OC(=O)C(O)c1ccccc1)C2,0.0
329,C=C1c2cccc(O)c2C(=O)C2C(=O)C3(O)C(=O)C(C(N)=O)...,1.0
330,OCC(O)c1oc(O)c(O)c1O,0.0


In [ ]:
seedup = pd.concat([dilidf2std,tdcdf2std]).drop_duplicates(keep=False)
seedup

,smiles,activity
0,OC[C@H]1O[C@H](S)[C@H](O)[C@@H](O)[C@@H]1O,1.0
1,CN(CCCCCCCCCCN(C)C(=O)Oc1cccc([N+](C)(C)C)c1)C...,1.0
2,COc1ccc2c(c1)[C@]13CCCC[C@@H]1[C@H](C2)N(C)CC3,0.0
3,CN1CC(C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4C(=O...,0.0
4,C[C@H]1C[C@H]2[C@@H]3CC=C4CC(=O)C=C[C@]4(C)[C@...,0.0
...,...,...
327,NC1CCCCC1N,1.0
328,C[N+]1(C)C2CCC1CC(OC(=O)C(O)c1ccccc1)C2,0.0
329,C=C1c2cccc(O)c2C(=O)C2C(=O)C3(O)C(=O)C(C(N)=O)...,1.0
330,OCC(O)c1oc(O)c(O)c1O,0.0


In [ ]:
zairachemdt['smiles']=

In [ ]:
dilies.to_csv('smilesandactivity2.csv',index=False)